In [ ]:
import csv
import ijson
from datetime import datetime
import numpy as np
import pandas as pd


## import data

In [ ]:
with open('data/part_v003_o001_r_00000.json', 'r') as file:
    pet_like = ijson.items(file, 'text', multiple_values=True)
    for value in pet_like:
        print(value)

In [25]:
from datetime import datetime
date_dt1 = datetime.strptime(date_dt, '%a %b %d %H:%M:%S +0000 %Y')
print(date_dt1)

2020-02-29 14:57:32


In [59]:
# extract 'dates'
start = time.time()
data = ijson.parse(open('../data/part_v003_o001_r_00000.json', 'r'), multiple_values=True)

l_date = [datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y') for prefix, event, value in data if prefix == 'created_at']

print ("elapsed_time:{0}".format(time.time() - start) + "[sec]")
slack_notify()

elapsed_time:1077.0222840309143[sec]


In [36]:
# extract 'tweets'
start = time.time()
data = ijson.parse(open('../data/part_v003_o001_r_00000.json', 'r'), multiple_values=True)

l_text = [datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y') for prefix, event, value in data if prefix == 'created_at']

print ("elapsed_time:{0}".format(time.time() - start) + "[sec]")


elapsed_time:1124.0678129196167[sec]


In [60]:
print(len(l_date) == len(l_text))

2057685 2057685


## test 

In [65]:
date, twt = l_date[:100], l_text[:100]

In [66]:
a = np.vstack([date, twt]).T

In [268]:
df = pd.DataFrame(a,columns=['date', 'tweet'])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
# RT_flag: True if tweet containt 'RT', otherwise False
df['RT_flag'] = df['tweet'].str.contains('RT')

# df

In [269]:
# divide df into one with and without RT
dfrt = df[df['RT_flag'] == True]
dfor = df[df['RT_flag'] == False]

In [270]:
# # info of dataframe
# print(dfor.info(), 'length', len(dfor))
# print('           ')
# print(dfrt.info(), 'length', len(dfrt))

In [271]:
# # save to csv
# dfrt.to_csv('../dfs/df_rt_100.csv', index=False)
# dfor.to_csv('../dfs/df_or_100.csv', index=False)
# dfrt.to_csv('../dfs/df_100.csv', index=False)

## Use dfor(DataFrame without RT) as a default 

In [344]:
# check dataframe's nan value
dfor.isnull().values.any()

False

In [343]:
words = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
words.columns

Index(['types', 'file_name', 'orignal_form', 'English_translation'], dtype='object')

In [351]:
print('words types:',set(words.types.tolist()))
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']

words types: {nan, '#Fatigue', '#Depression', '#Vigor', '#Tension', '#Confusion', '#Anger'}


## DataFrame of each filetypes

In [422]:
# Drop nan and False in order to use query 
booleanDictionary = {True: 'TRUE', False: 'FALSE'}
words = words.replace(booleanDictionary)
words = words.dropna(how='all')

df_c = words.query('file_name.str.contains("D")', engine='python')
df_c.head()

,types,file_name,orignal_form,English_translation
22,#Depression,D1,孤独,loneliness
23,NaN,D2,不幸だ,unhappy
24,NaN,D3,悲しく,sadly
25,NaN,D4,憂鬱,melancholy
26,NaN,D5,がっかり,disappointed


## extract data which includee keyword

In [425]:
keyword_jp = df_c.orignal_form.tolist()
df_d_query = dfor[dfor['tweet'].str.contains('|'.join(keyword_jp))]
df_d_query.head()

,date,tweet,year,month,day,hour,RT_flag
2,2020-02-29 14:59:58,長期的にはジャイロに慣れた方が良いんだけど、今回の大会しか見てないのかなってちょっと悲しくなった,2020,2,29,14,False
5,2020-02-29 14:59:58,フェルナンドと遊ぼう\n・華麗なるフェルナンド\n・フェルナンドの憂鬱\n・翔べフェルナンド...,2020,2,29,14,False
10,2020-02-29 14:59:58,@takukuroneko BLEEDは1の哀しいストーリーを見事に表現してるところが良いで...,2020,2,29,14,False
16,2020-02-29 14:59:58,最近まためみたんロスが来ててこの曲聞くとなんか悲しくなる https://t.co/yuaW...,2020,2,29,14,False
36,2020-02-29 14:59:56,@irohasujo_ 寂しがり屋で構ってちゃんでしょうか……\nそれに値しないことすら気づ...,2020,2,29,14,False


In [284]:
l1 = '@ss_ars_42 ちょっと面白すぎて笑ったwwwww\nくらこかわいいねえ！！！！！！！'
l2 = ('面白')
l3 = [0, 1, 5]
l4 = [5, 6, 7]

print(set(l2) <= set(l1))
set(l1).isdisjoint(set(l4))

True


True

In [ ]:
# words
# words_types = [x for x in  words.types.unique() if x == x]
# filenames = [x for x in  words['file_name'].unique() if x == x]
# oforms = [x for x in  words['orignal_form'].unique() if x == x]

In [ ]:
dfor.query('tweet.str.contains("です|ジャイロ|w|富野")', engine='python')